In [ ]:
import os
import numpy as np 
from astropy.io import fits 
import matplotlib.pyplot as plt 
import py_specrebin_vec
from matplotlib import rc
import pandas as pd
from scipy import optimize
from scipy import integrate
path_name = '.'

Issues w/ Mask:A2M33P Slit:181, Mask:B2M33P Slit:139

RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 800.


## Variable Pannel

In [ ]:
mask_name = "M33D2A" #Enter your mask here!
grating = 600 #Keep the grating 600. BPT doesn't use 1200 grating.

## Define Wavelength

In [ ]:
new_wave_600 = np.arange(4000, 11000, .65) 
new_wave_1200 = np.arange(6000, 11000, .33) 

## Read Data

In [ ]:
Data = fits.open("./AGST Subtracted Spectra/{}_AGST_Subtracted_Spectra.fits.gz".format(mask_name))
AGST_RBFlux = Data[1].data #Read and extract data

## Function To Read ISM_EM_LINES.txt & Extract Slit # of Excluded Slits

In [ ]:
def get_exclusions():
    filepath = 'ISM_EM_LINES.txt'
    fp = open(filepath)
    all_data = []
    for line in (fp):
        mask_name = line.split(':')[0].split('_')[0]
        slit_number = line.split(':')[1].strip().split(" ")[0]
        if len(slit_number) == 2:
            slit_number = '0' + slit_number
        elif len(slit_number) == 1:
            slit_number = '00' + slit_number
        else:
            pass
        object_id = line.split(':')[1].strip().split()[1]
        data = {}
        data['mask_name'] = mask_name
        data['slit_number'] = slit_number
        data['object_id'] = object_id
        all_data.append(data)
    return all_data     

In [ ]:
def get_files_to_include(folder):
    import os
    list_of_files_to_include = []
    list_of_files_to_exclude = []
    serendip_files = []
    all_file_names_in_folder = os.listdir('data/{}'.format(folder))
    y = len(all_file_names_in_folder)
    print("The number of files in the folder is {0}".format(y))
    all_data = get_exclusions()
    len_all_data = len(all_data)
    for n in range(y):
        parts_of_file_name = all_file_names_in_folder[n].split(".")
        if parts_of_file_name[0] == 'spec1d': # avoids hidden DS_Store files on my mac
            object_id = parts_of_file_name[3]
            slit_number = parts_of_file_name[2]
            mask_name = parts_of_file_name[1]
            should_include = True
            should_exclude = True
            for k in range(len_all_data):
                if ((object_id == all_data[k]['object_id']) and (slit_number == all_data[k]['slit_number']) and (mask_name == all_data[k]['mask_name'])):
                    should_include = False
                    should_exclude = True
                if 'serendip' in object_id:
                    should_include = False
                    should_exclude = False
            if should_include == True:
                list_of_files_to_include.append(all_file_names_in_folder[n])       
            elif should_exclude == True:
                list_of_files_to_exclude.append(all_file_names_in_folder[n])
            elif should_include == False & should_exclude == False:
                serendip_files.append(all_file_names_in_folder[n])
    
    print('The number of files left after exclusions is {0}'.format(len(list_of_files_to_include)))
    
    return sorted(list_of_files_to_include), sorted(list_of_files_to_exclude), sorted(serendip_files)


In [ ]:
def get_slit_nums(files):
    
    slit_nums = []
    
    if len(files) > 1:
    
        for i in range(len(files)):
            parts_of_file_name = files[i].split(".")
            slit_num = parts_of_file_name[2]
            slit_nums.append(int(slit_num))
            
    return slit_nums

## Calls To Get Slit Numbers

In [ ]:
#filtering files
list_of_files_to_include, list_of_files_to_exclude, list_of_serendip_files = get_files_to_include(mask_name)

file_names = list_of_files_to_include
file_names_exclude = list_of_files_to_exclude
file_names_serendip = list_of_serendip_files
file_names_all = list_of_files_to_include + list_of_files_to_exclude

In [ ]:
slit_nums = get_slit_nums(file_names) #get slit # of INCLUDED slits
slit_nums_exclude = get_slit_nums(file_names_exclude) #get slit # of EXCLUDED slits
all_slit_nums = get_slit_nums(file_names_all) #slit # of INCLUDED & EXCLUDED slits

print("Slit # to INCLUDE in median calculation: {0}".format(slit_nums))
print("Slit # to EXCLUDE: {0}".format(slit_nums_exclude))

## Read From Text File

In [ ]:
def Slits_For_BPT(mask):
    
    '''
    Return slit number of slits that are used to create BPT diagrams. 
    
    Parameters
    ---
    mask: str
        Name of mask.
        
        
    Returns
    ---
    slit_nums: list
        List containing slit number of slits in input mask that are used to create BPT diagram. 
        
    '''
    
    txtfile = open("BPT_EM_LINES.txt","r") #Opening and reading the text file. 
    lines = txtfile.readlines()

    all_mask_names = [] #empty list to add into 
    all_slit_nums = [] #empty list to add into

    for line in lines: 
        ind_element = line.split(".") #Split into three elements.
        all_mask_names.append(ind_element[0]) #0th element contains mask name
        all_slit_nums.append(ind_element[1]) #1st element contains slit number

    slit_nums = np.array(all_slit_nums)[np.array(all_mask_names) == "{}".format(mask)] #Create boolean using mask name and use boolean to index.
    slit_nums = [int(val) for val in slit_nums] #Convert str -> int.
    
    if len(slit_nums) == 0:
        print("No Slits Use in BPT Diagram. Move to Next Mask.")
        raise KeyboardInterrupt("No Slits Here! Move to Next Mask!")
    else:
        pass
    
    return slit_nums 

In [ ]:
SlitNums_of_Mask = Slits_For_BPT(mask_name)

## Gaussian Integral Function

In [ ]:
def gaussian(x, amplitude, mean, stddev):
    return amplitude*np.exp(-((x - mean)**2/(2*(stddev**2))))

In [ ]:
def area(window_min,window_max,addons_window_pixels,flux,*args,wave=new_wave_600):
    
    '''
    Calculate the emission line strength of a single emission line. Set first window to 
    search for emission line within the window. Second window capture data points within first window.
    Use captured data to determine Gaussian parameters and performs Gaussian fit. 

    
    Parameters
    ---
    window_min: int or float
        Minimal value of 1st window (search window). 
    
    window_max: int or float
        Maximal value of 1st window (search window).
    
    addons_window_pixels: int
        Number of pixels to add onto center of emission line. Width of 2nd window = 2x addons_window_pixels
    
    flux: ndarry
        Flux of one slit.
    
    *arg: 1.int 2.str
        1. Slit number.
        2. Name of emission line.
    
    wave: ndarray
        Wavelength of one slit.
    
    
    Returns
    ---
    Line_STR: int or float
        Calculated strength of an emission line.
    
    '''
    
    #1st window (Search window)
    WavWin_Bool = (wave > window_min) & (wave < window_max) #Boolean array of wavelength between two set limits
    Window_X = wave[WavWin_Bool] #Use boolean array to index x-values of first window
    Window_Y = flux[WavWin_Bool] #Use boolean array to index y-values of first window
    
    #2nd window (Capture window)
    Peak_Idx = list(flux).index(max(Window_Y)) #Determine the index of the peak
    New_Window_Lim = np.array([Peak_Idx-addons_window_pixels,Peak_Idx+addons_window_pixels]) #Set pixel length of second window
    Wavelength = wave[New_Window_Lim[0]:New_Window_Lim[1]] #Wavelength of second window
    Flux = flux[New_Window_Lim[0]:New_Window_Lim[1]] #Flux of second window
    
    #Calculate Amplitude, Mean, and Standard Dev
    p0_A = max(Flux) #Guess Amplitude
    p0_mu = Wavelength[list(Flux).index(p0_A)] #Guess Mean
    p0_sigma = np.std(Wavelength) #Guess Standard Dev
    popt,_ = optimize.curve_fit(gaussian,Wavelength,Flux,p0=[p0_A,p0_mu,p0_sigma])
    
    #Calculate line strength
    Amplitude = popt[0]
    STD = popt[2]
    Line_STR = abs(Amplitude * STD) #Using the Amp*SD/0.3989 formula. Ratio ignore constant.
    
    #Visual inspection
    gaussian_x = np.linspace(wave[New_Window_Lim[0]],wave[New_Window_Lim[1]],100)
    gaussian_y = gaussian(gaussian_x,popt[0],popt[1],popt[2])
    plt.plot(Window_X,Window_Y,color="black",label="First Window")
    plt.plot(Wavelength,Flux,color="blue",label="Second Window")
    plt.plot(gaussian_x,gaussian_y,color="red",linestyle="--",label="Gaussian")
    plt.vlines(wave[New_Window_Lim[0]],-500,1000,linestyle="--",color="green",label="2nd Window Limit")
    plt.vlines(wave[New_Window_Lim[1]],-500,1000,linestyle="--",color="green",label="2nd Window Limit")
    plt.xlim(window_min,window_max)
    plt.ylim(-30,Amplitude+50)
    plt.title("Slit Number:{} EM Line:{}".format(args[0],args[1]))
    plt.xlabel("Wavelength")
    plt.ylabel("Flux (e/hr)")
    plt.legend()
    plt.show()
    
    #Print line strength
    print("Amplitude: {}".format(Amplitude))
    print("Standard Deviation: {}".format(STD))
    print("Slit: {} {} Line Strength: {}".format(args[0],args[1],Line_STR))
    
    return Line_STR

In [ ]:
def integral_and_ratios(SlitNums_Use,SlitNums_All,AGST_Sub_Data):
    
    OIII_Hb_Ratio_List = [] 
    NII_Ha_Ratio_List = []
    Hb_Line_STR_List = []
    OIII_Line_STR_List = []
    Ha_Line_STR_List = []
    NII_Line_STR_List = []
    Idx_List = []
    
    for slit_number in SlitNums_Use:
        
        Idx_of_Slit = SlitNums_All.index(slit_number) #Extract AGST data
        Idx_List.append(Idx_of_Slit)
        Data_of_Slit = AGST_Sub_Data[Idx_of_Slit]
        
        Hb_Line_STR = area(4840,4940,15,Data_of_Slit,slit_number,"H Beta") #Calculate the line strength of the H Beta emission line
        Hb_Line_STR_List.append(Hb_Line_STR)
        
        OIII_Line_STR = area(4940,5040,15,Data_of_Slit,slit_number,"OIII") #Calculate the line strength of the OIII emission line
        OIII_Line_STR_List.append(OIII_Line_STR)
        
        Ha_Line_STR = area(6540,6570,15,Data_of_Slit,slit_number,"H Alpha") #Calculate the line strength of the H Alpha emission line
        Ha_Line_STR_List.append(Ha_Line_STR)
        
        NII_Line_STR = area(6570,6600,15,Data_of_Slit,slit_number,"NII") #Calculate the line strength of the NII emission line
        NII_Line_STR_List.append(NII_Line_STR)
        
        
        
        
        OIII_Hb_Ratio = OIII_Line_STR/Hb_Line_STR #Calculate OIII/Hb ratio
        OIII_Hb_Ratio_List.append(OIII_Hb_Ratio)
        
        NII_Ha_Ratio = NII_Line_STR/Ha_Line_STR #Calculate NII/Ha ratio
        NII_Ha_Ratio_List.append(NII_Ha_Ratio)
        
    All_Returned_Data = {"OIII/Hb Ratio":OIII_Hb_Ratio_List,"NII/Ha Ratio":NII_Ha_Ratio_List,
                        "Hb":Hb_Line_STR_List,"OIII":OIII_Line_STR_List,"Ha":Ha_Line_STR_List,
                        "NII":NII_Line_STR_List}
    
    return All_Returned_Data,Idx_List
        

In [ ]:
Returned_Data,Idx_List = integral_and_ratios(SlitNums_of_Mask,slit_nums_exclude,AGST_RBFlux)

In [ ]:
Returned_Data

## Extract Redshift, QOP, and Create DF

In [ ]:
import numpy
def loadMarzResults(filepath):
    return numpy.genfromtxt(filepath, delimiter=',', skip_header=2, autostrip=True, names=True, dtype=None)

In [ ]:
#arrays containing all slits processed in MARZ
res = loadMarzResults("./Marz_Results (Redone)/{0}_Marz_KN.mz".format(mask_name)) 
confident = res[res['QOP'] == 4]

In [ ]:
redshift = [] #empty list for redshift value
QOP = [] #empty list for QOP

for n in range(len(res)): #sorting 
    redshift.append(res[n][12])
    QOP.append(res[n][13])

## Add Redshift to CSV File

In [ ]:
df_1= pd.DataFrame({"Mask Name":np.full(len(SlitNums_of_Mask),mask_name), #add mask name to DF
                   "Slit Numbers":np.array(SlitNums_of_Mask),                     
                   "Redshift":np.array(redshift)[np.array(Idx_List)], #add redshift to DF
                   "QOP":np.array(QOP)[np.array(Idx_List)]}) #add QOP to DF

df_2 = pd.DataFrame(Returned_Data)

df = pd.concat([df_1,df_2],axis=1)

if os.path.isfile("./BPT Diagram CSV/BPT_Ratios_Redo.csv") == False: #check if file exist. If not make it.
    df.to_csv("./BPT Diagram CSV/BPT_Ratios_Redo.csv",index=False) 
    
else: #if file does exist, open and add to it.
    redshift_read = pd.read_csv("./BPT Diagram CSV/BPT_Ratios_Redo.csv")
    new_csv = pd.concat([redshift_read,df])
    new_csv.to_csv("./BPT Diagram CSV/BPT_Ratios_Redo.csv",index=False)


## Read CSV

In [ ]:
Final_DF = pd.read_csv("./BPT Diagram CSV/BPT_Ratios_Redo.csv")
Median_of_Ha = np.median(Final_DF["Ha"])

In [ ]:
Log_of_NII_Ha_Ratio = np.log10(Final_DF["NII/Ha Ratio"])
Log_of_OIII_Hb_Ratio = np.log10(Final_DF["OIII/Hb Ratio"])
plt.scatter(Log_of_NII_Ha_Ratio[Final_DF["Ha"]>Median_of_Ha],Log_of_OIII_Hb_Ratio[Final_DF["Ha"]>Median_of_Ha],label="Sorted by Strong Ha",s=10)
plt.scatter(Log_of_NII_Ha_Ratio[Final_DF["Ha"]<Median_of_Ha],Log_of_OIII_Hb_Ratio[Final_DF["Ha"]<Median_of_Ha],label="Sorted by Weak Ha",s=10)
plt.xlabel("$Log_{10}([NII]/Hα)$")
plt.ylabel("$Log_{10}([OIII]/Hβ)$")
plt.title("BPT Diagram")
plt.xlim(-1.5,0)
plt.ylim(-2.5,1)
plt.legend()
plt.grid(True)
plt.show()
#plt.yscale('log')
#plt.xscale('log')

In [ ]:
NII_Ha_Ratio = Final_DF["NII/Ha Ratio"]
OIII_Hb_Ratio = Final_DF["OIII/Hb Ratio"]
plt.scatter(NII_Ha_Ratio[Final_DF["Ha"]>Median_of_Ha],Log_of_OIII_Hb_Ratio[Final_DF["Ha"]>Median_of_Ha],label="Sorted by Strong Ha",s=10)
plt.scatter(NII_Ha_Ratio[Final_DF["Ha"]<Median_of_Ha],Log_of_OIII_Hb_Ratio[Final_DF["Ha"]<Median_of_Ha],label="Sorted by Weak Ha",s=10)
plt.xlabel("[NII]/Hα")
plt.ylabel("[OIII]/Hβ")
plt.title("BPT Diagram")
#plt.xlim(-1.5,0)
#plt.ylim(-2.5,1)
plt.xlim(0.03,3)
plt.ylim(0.07,12)
plt.legend()
plt.grid(True)
plt.yscale('log')
plt.xscale('log')
plt.show()


## Adding Reference Lines

In [ ]:
def ref_curve(NII_Ha_RefVal):
    return 10**(1.10-(0.6/(0.01-np.log10(NII_Ha_RefVal))))

In [ ]:
NII_Ha_RefVal = np.arange(0,1,0.01)
OIII_Hb_RefVal = ref_curve(NII_Ha_RefVal)

In [ ]:
NII_Ha_Ratio = Final_DF["NII/Ha Ratio"]
OIII_Hb_Ratio = Final_DF["OIII/Hb Ratio"]
plt.scatter(NII_Ha_Ratio[Final_DF["Ha"]>Median_of_Ha],Log_of_OIII_Hb_Ratio[Final_DF["Ha"]>Median_of_Ha],label="Sorted by Strong Ha",s=10)
plt.scatter(NII_Ha_Ratio[Final_DF["Ha"]<Median_of_Ha],Log_of_OIII_Hb_Ratio[Final_DF["Ha"]<Median_of_Ha],label="Sorted by Weak Ha",s=10)
plt.plot(NII_Ha_RefVal,OIII_Hb_RefVal,color="grey",linestyle="--")
plt.xlabel("[NII]/Hα")
plt.ylabel("[OIII]/Hβ")
plt.title("BPT Diagram")
#plt.xlim(-1.5,0)
#plt.ylim(-2.5,1)
plt.xlim(0.03,3)
plt.ylim(0.07,12)
plt.legend()
#plt.grid(True)
plt.yscale('log')
plt.xscale('log')
plt.show()


In [ ]:
def K03(NII_Ha_RefVal):
    return 10**((0.61/(np.log10(NII_Ha_RefVal)-0.05))+1.30)

In [ ]:
NII_Ha_K03 = np.arange(0,1,0.01)
OIII_Hb_K03 = K03(NII_Ha_K03)

In [ ]:
NII_Ha_Ratio = Final_DF["NII/Ha Ratio"]
OIII_Hb_Ratio = Final_DF["OIII/Hb Ratio"]
plt.scatter(NII_Ha_Ratio[Final_DF["Ha"]>Median_of_Ha],Log_of_OIII_Hb_Ratio[Final_DF["Ha"]>Median_of_Ha],label="Sorted by Strong Ha",s=10)
plt.scatter(NII_Ha_Ratio[Final_DF["Ha"]<Median_of_Ha],Log_of_OIII_Hb_Ratio[Final_DF["Ha"]<Median_of_Ha],label="Sorted by Weak Ha",s=10)
plt.hlines(3,0.03,3,linewidth=0.5,color="grey")
plt.vlines(0.6,0.01,20,linewidth=0.5,color="grey")
plt.plot(NII_Ha_RefVal,OIII_Hb_RefVal,color="grey",linestyle="--")
plt.plot(NII_Ha_K03,OIII_Hb_K03,color="grey",linestyle="--")
plt.xlabel("[NII] $\lambda$6583/Hα")
plt.ylabel("[OIII] $\lambda$5007/Hβ")
plt.title("BPT Diagram")
plt.text(1,0.5,"LINERs",fontsize=16)
plt.text(1,10,"Seyferts",fontsize=16)
plt.text(0.04,1,"Star-forming galaxies",fontsize=12)
plt.annotate("Reference curve",xy=(0.2,2),xytext=(0.3,7),arrowprops=dict(facecolor="black",width=1.5,headwidth=7))
plt.annotate("K03",xy=(0.3,2),xytext=(0.5,2),arrowprops=dict(facecolor="black",width=1.5,headwidth=7))
plt.xlim(0.03,3)
plt.ylim(0.01,20)
plt.legend(loc="upper left")
plt.yscale('log')
plt.xscale('log')
plt.show()


In [ ]:
def EmRatio_VarTemp_OIII_Hb(xi,T_4):
        return 214*xi*T_4**(0.494+(0.089*np.log(T_4)))*np.exp(-2.917/T_4)*(10**(8.69-12)/5.37e-4)

In [ ]:
def EmRatio_VarTemp_NII_Ha(xi,T_4):
    return 12.4*(1-xi)*T_4**(0.495+(0.04*np.log(T_4)))*np.exp(-2.204/T_4)*(10**(7.83-12)/7.41e-5)

In [ ]:
Xi = np.arange(0,1.01,0.01)

OIII_Hb_7K = EmRatio_VarTemp_OIII_Hb(Xi,0.7)
NII_Ha_7K = EmRatio_VarTemp_NII_Ha(Xi,0.7)

OIII_Hb_8K = EmRatio_VarTemp_OIII_Hb(Xi,0.8)
NII_Ha_8K = EmRatio_VarTemp_NII_Ha(Xi,0.8)

OIII_Hb_9K = EmRatio_VarTemp_OIII_Hb(Xi,0.9)
NII_Ha_9K = EmRatio_VarTemp_NII_Ha(Xi,0.9)

In [ ]:
NII_Ha_Ratio = Final_DF["NII/Ha Ratio"]
OIII_Hb_Ratio = Final_DF["OIII/Hb Ratio"]
plt.scatter(NII_Ha_Ratio[Final_DF["Ha"]>Median_of_Ha],Log_of_OIII_Hb_Ratio[Final_DF["Ha"]>Median_of_Ha],label="Sorted by Strong Ha",s=10)
plt.scatter(NII_Ha_Ratio[Final_DF["Ha"]<Median_of_Ha],Log_of_OIII_Hb_Ratio[Final_DF["Ha"]<Median_of_Ha],label="Sorted by Weak Ha",s=10)
plt.hlines(3,0.03,3,linewidth=0.5,color="black")
plt.vlines(0.6,0.01,20,linewidth=0.5,color="black")
plt.plot(NII_Ha_RefVal,OIII_Hb_RefVal,color="black",linestyle="--",linewidth=0.7,dashes=(6,8))
plt.plot(NII_Ha_K03,OIII_Hb_K03,color="black",linestyle="--",linewidth=0.7,dashes=(6,8))
plt.plot(NII_Ha_7K,OIII_Hb_7K,color='black',linewidth=0.7)
plt.text(0.035,7,"9000 K",fontsize=12)
plt.plot(NII_Ha_8K,OIII_Hb_8K,color='black',linewidth=0.7)
plt.text(0.035,4.5,"8000 K",fontsize=12)
plt.plot(NII_Ha_9K,OIII_Hb_9K,color='black',linewidth=0.7)
plt.text(0.035,2.3,"7000 K",fontsize=12)
plt.xlabel("[NII] $\lambda$6583/Hα",fontsize=16)
plt.ylabel("[OIII] $\lambda$5007/Hβ",fontsize=16)
plt.title("BPT Diagram")
plt.text(1,0.5,"LINERs",fontsize=16)
plt.text(1,4,"Seyferts",fontsize=16)
plt.text(0.035,1,"Star-forming galaxies",fontsize=12)
plt.annotate("Reference curve",xy=(0.2,2),xytext=(0.12,10.7),arrowprops=dict(facecolor="black",width=1.5,headwidth=7),fontsize=11)
plt.annotate("K03",xy=(0.3,2),xytext=(0.4,6),arrowprops=dict(facecolor="black",width=1.5,headwidth=7),fontsize=11)
plt.xlim(0.03,3)
plt.ylim(0.06,20)
plt.legend(loc="upper right",bbox_to_anchor=(1.2,1))
plt.yscale('log')
plt.xscale('log')
plt.savefig("BPT.png")
plt.show()

## Error Calculation

In [ ]:
bool_idx = (new_wave_600 > 5735) & (new_wave_600 < 5765)

In [ ]:
plt.plot(new_wave_600[bool_idx],AGST_RBFlux[2][bool_idx])
wave_diff = np.diff(new_wave_600[bool_idx])
delta_EW = np.sum(abs(wave_diff * AGST_RBFlux[2][bool_idx][0:len(wave_diff)]))
delta_EW

In [ ]:
Linear_Func = lambda x,m,b: (m*x)+b
Y_Intercept_Func = lambda x,y,m: y-(m*x)
X_Value_Func = lambda y,m,b: (y-b)/m

In [ ]:
def err_calculation(Wave,AGST_Flux):
    
    tot_area = 0
    plt.figure()
    plt.plot(Wave,AGST_Flux)
    
    for idx in range(len(AGST_Flux)-1):
        
        if AGST_Flux[idx]>0 and AGST_Flux[idx+1]>0:
            m = (AGST_Flux[idx+1]-AGST_Flux[idx])/(Wave[idx+1]-Wave[idx])
            b = Y_Intercept_Func(Wave[idx],AGST_Flux[idx],m)
            area = integrate.quad(Linear_Func,Wave[idx],Wave[idx+1],args=(m,b))
            print(area[0])
            plt.fill_between(np.array([Wave[idx],Wave[idx+1]]),np.array([AGST_Flux[idx],AGST_Flux[idx+1]]),0)
            tot_area += area[0]
            
        elif AGST_Flux[idx]>0 and AGST_Flux[idx+1] < 0:
            m = (AGST_Flux[idx+1]-AGST_Flux[idx])/(Wave[idx+1]-Wave[idx])
            b = Y_Intercept_Func(Wave[idx],AGST_Flux[idx],m)
            x_val_at_0 = X_Value_Func(0,m,b)
            area = integrate.quad(Linear_Func,Wave[idx],x_val_at_0,args=(m,b))
            print(area[0])
            plt.fill_between(np.array([Wave[idx],x_val_at_0]),np.array([AGST_Flux[idx],Linear_Func(x_val_at_0,m,b)]),0)
            tot_area += area[0]
            
        elif AGST_Flux[idx]<0 and AGST_Flux[idx+1]>0:
            m = (AGST_Flux[idx+1]-AGST_Flux[idx])/(Wave[idx+1]-Wave[idx])
            b = Y_Intercept_Func(Wave[idx],AGST_Flux[idx],m)
            x_val_at_0 = X_Value_Func(0,m,b)
            area = integrate.quad(Linear_Func,x_val_at_0,Wave[idx+1],args=(m,b))
            print(area[0])
            plt.fill_between(np.array([x_val_at_0,Wave[idx+1]]),np.array([Linear_Func(x_val_at_0,m,b),AGST_Flux[idx+1]]),0)
            tot_area += area[0]
            
        elif AGST_Flux[idx]<0 and AGST_Flux[idx+1]<0:
            pass
    plt.show()    
    return tot_area
        

In [ ]:
err = err_calculation(new_wave_600[bool_idx],AGST_RBFlux[2][bool_idx])

In [ ]:
err